# Chuyển đổi file parquet sang csv


In [ ]:
from scipy.stats import spearmanr, rankdata
from scipy.spatial.distance import pdist, squareform

import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import shutil
import time
import _pickle as cp
import sys
from astropy.stats import median_absolute_deviation
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MinMaxScaler

from PIL import Image

In [ ]:
dataset = 'CIC-DDoS2019'

In [ ]:
paths = [f'./datasets/{dataset}/parquet/LDAP-training.parquet',
         f'./datasets/{dataset}/parquet/MSSQL-training.parquet',
         f'./datasets/{dataset}/parquet/NetBIOS-training.parquet',
         f'./datasets/{dataset}/parquet/Portmap-training.parquet',
         f'./datasets/{dataset}/parquet/Syn-training.parquet',
         f'./datasets/{dataset}/parquet/UDP-training.parquet',
         f'./datasets/{dataset}/parquet/UDPLag-training.parquet']

In [ ]:
# Bước 1: Đọc dữ liệu từ các file .parquet
dfs = []
for file in paths:
    try:
        df = pd.read_parquet(file)
        dfs.append(df)
        print(f"Đã đọc thành công: {file}")
    except Exception as e:
        print(f"Lỗi khi đọc file {file}: {e}")

Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/LDAP-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/MSSQL-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/NetBIOS-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/Portmap-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/Syn-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/UDP-training.parquet
Đã đọc thành công: ./datasets/CIC-DDoS2019/parquet/UDPLag-training.parquet


In [ ]:
# Bước 2: Kiểm tra tính đồng nhất của các đặc trưng (columns)
# Lấy cột của bộ dữ liệu đầu tiên làm chuẩn
base_columns = dfs[0].columns

all_columns_same = True

for i, df in enumerate(dfs):
    if not df.columns.equals(base_columns):
        print(f"File {paths[i]} có các đặc trưng khác biệt. Hủy quá trình hợp nhất.")
        all_columns_same = False
        break

In [ ]:
# chuyển tập dữ liệu parquet thành csv
csv_folder_path = f'./datasets/{dataset}/csv'
os.makedirs(csv_folder_path, exist_ok=True)
for path in paths:

    arr = path.split('/')
    file_name = arr[-1].split('.')[0] + '.csv'

    # Đọc file Parquet
    df = pd.read_parquet(path, engine='pyarrow')

    # Lưu dưới dạng CSV
    df.to_csv(f'./datasets/{dataset}/csv/' + file_name, index=False)

In [ ]:
# Hợp nhất dữ liệu nếu tất cả các bộ dữ liệu có cùng đặc trưng
if all_columns_same:
    # Sử dụng hàm concat để hợp nhất
    combined_df = pd.concat(dfs, ignore_index=True)
    print("Hợp nhất thành công các bộ dữ liệu.")

    # Bước 4: Lưu dữ liệu hợp nhất
    # Lưu file dưới dạng .parquet
    # parquet_output_path = './data/combine data/data.parquet'
    # combined_df.to_parquet(parquet_output_path)
    # print(f"Lưu file .parquet thành công tại: {parquet_output_path}")

    # Xuất dữ liệu ra file CSV
    csv_output_path = f'./datasets/{dataset}/csv/_combine_data.csv'
    combined_df.to_csv(csv_output_path, index=False)
    print(f"Lưu file .csv thành công tại: {csv_output_path}")
else:
    print("Hủy quá trình hợp nhất do sự khác biệt về đặc trưng giữa các bộ dữ liệu.")

Hợp nhất thành công các bộ dữ liệu.
Lưu file .csv thành công tại: ./datasets/CIC-DDoS2019/csv/_combine_data.csv
